# Testbench File

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder

from qiskit_metal.qlibrary.passives.cap_three_fingers import CapThreeFingers
from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond

In [4]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [5]:
design.chips.main.size['size_x'] = '9mm'
design.chips.main.size['size_y'] = '3mm'
# wide cpw 
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um' 

design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '3mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [6]:
## Basic operation
options = dict(
    pad_width = '450 um',
    pad_height= '100 um',
    pad_gap = '30um',
    
    pocket_height = '650 um',
    cl_pocket_edge = '180',
    connection_pads = dict(
        R = dict(loc_W=+1, loc_H=+1, pocket_rise = '0um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um'),
        L = dict(loc_W=-1, loc_H=-1, pocket_rise = '0um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um')
    ))

Q1 = TransmonPocket(design,'Q1', options = dict(
        pos_x='-1.2mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '11.8nH',
        q3d_inductance =  '11.8nH',
        **options))

Q2 = TransmonPocket(design,'Q2', options = dict(
        pos_x='+1.2mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '10.9nH',
        q3d_inductance =  '10.9nH',
        **options))

gui.rebuild()
gui.autoscale()

In [7]:
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

RouteMeander.get_template_options(design)


options = Dict(
        fillet = '90um',
        lead=Dict(
            start_straight='100um',
            end_straight='100um'),
        trace_gap='9um',
        trace_width='15um')

asym = -50
cpw1 = connect('cpw1', 'Q1', 'R', 'Q2', 'L', '8.4 mm', f'+{asym}um')

gui.rebuild()
gui.autoscale()

In [8]:
cap_Q1 = CapThreeFingers(design, 'Cap_Q1', options = dict(pos_x = '-3.2 mm',
                                                          pos_y = '0.5mm',
                                                         orientation = '90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))
cap_Q2 = CapThreeFingers(design, 'Cap_Q2', options = dict(pos_x = '+3.2 mm',
                                                          pos_y = '-0.5 mm',
                                                         orientation = '-90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))


gui.rebuild()
gui.autoscale()

In [9]:
RO_Q1 = RouteMeander(design, 'RO_Q1', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q1',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q1',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.5 mm'))

RO_Q2 = RouteMeander(design, 'RO_Q2', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q2',pin = 'R'),
                              end_pin = Dict(component = 'Cap_Q2',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.3 mm'))

gui.rebuild()
gui.autoscale()

In [44]:
## 여기까지 geometry

## 2. Render into Ansys Q3D

In [10]:
fourq_q3d = design.renderers.q3d

Skip the next cell if Ansys is already open and with a project loaded

In [14]:
#fourq_q3d.open_ansys()
fourq_q3d.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM21.1\Win64')

**Wait for Ansys to open up completely before proceeding to the next cell**

In [11]:
fourq_q3d.connect_ansys()
#fourq_q3d.add_q3d_design("Q3dMetalDesign")

INFO 10:38AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:38AM [load_ansys_project]: 	Opened Ansys App
INFO 10:38AM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 10:38AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/LeeJaeho/Documents/Ansoft/
	Project:   Project9
INFO 10:38AM [connect_design]: 	Opened active design
	Design:    Q3DDesign1 [Solution type: Q3D]
WARNING 10:38AM [connect_setup]: 	No design setup detected.
INFO 10:38AM [connect]: 	Connection to Ansys established successfully. 😀 



Render a single qubit with 2 endcaps

In [12]:
#fourq_q3d.render_design([], [])
#fourq_q3d.render_design(['Q2'], [('Q2', 'R'), ('Q2', 'L')])
fourq_q3d.render_design(['Q1'], [('Q1', 'R'), ('Q1', 'L')])

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix

In [13]:
#     Add a solution setup:
fourq_q3d.add_q3d_setup()
#     Example of how to change defaults (just an example. We will not use this setup):
#fourq_q3d.add_q3d_setup(name = 'Setup', max_passes = 20)

Optional keyword parameters and their defaults, listed below, represent parameters in this Q3D solution setup. Running fourq_q3d.add_q3d_setup() is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. To easily keep track of different solution setups, give each of them a different name, otherwise, Ansys will simply append numbers to the end of duplicate names. <br><br>
freq_ghz: float = 5. <br>
name: str = "Setup" <br>
save_fields: bool = False <br>
enabled: bool = True <br>
max_passes: int = 15 <br>
min_passes: int = 2 <br>
min_converged_passes: int = 2 <br>
percent_error: float = 0.5 <br>
percent_refinement: int = 30 <br>
auto_increase_solution_order: bool = True <br>
solution_order: str = 'High' <br>
solver_type: str = 'Iterative <br><br>
These defaults can be modified by including the keywords and their new values inside the parentheses. For example, to increase the maximum number of passes from the default value of 15 up to 20 while leaving all else unchanged, replace the cell above with the following: <br><br>
fourq_q3d.add_q3d_setup(max_passes = 20)

In [14]:
# Analyze said solution setup.
fourq_q3d.analyze_setup("Setup")

INFO 10:39AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:39AM [analyze]: Analyzing setup Setup


In [15]:
# Using the analysis results, get its capacitance matrix as a dataframe.
fourq_q3d.get_capacitance_matrix()

ground_main_plane  L_connector_pad_Q1  pad_bot_Q1  \
ground_main_plane           208.13586           -29.06592   -43.67895   
L_connector_pad_Q1          -29.06592            39.29862    -8.15558   
pad_bot_Q1                  -43.67895            -8.15558    86.77402   
pad_top_Q1                  -43.90126            -1.27262   -30.77767   
R_connector_pad_Q1          -28.89364            -0.08539    -1.21691   

                    pad_top_Q1  R_connector_pad_Q1  
ground_main_plane    -43.90126           -28.89364  
L_connector_pad_Q1    -1.27262            -0.08539  
pad_bot_Q1           -30.77767            -1.21691  
pad_top_Q1            86.90453            -8.01352  
R_connector_pad_Q1    -8.01352            38.93456

The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
pass_number: int = 3

In [19]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(11.8, 0, 2, 7.5, [6.5], 9)
#dict_lom = fourq_q3d.lumped_oscillator_vs_passes(10.9, 0, 2, 7.7, [6.5], 9)
dict_lom

Pass number:  1
[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.903517 [GHz]
EC 356.312793 [MHz]
EJ 13.847085 [GHz]
alpha -419.451115 [MHz]
dispersion 106.412352 [KHz]
Lq 0.000054 [nH]
Cq 54.362988 [fF]
T1 179.643119 [us]

**Coupling Properties**

tCqbus1 -3.483934 [fF]
gbus1_in_MHz -62.056667 [MHz]
χ_bus1 -1.022364 [MHz]
1/T1bus1 879.333017 [Hz]
T1bus1 180.995072 [us]

tCqbus2 0.409719 [fF]
gbus2_in_MHz 6.322633 [MHz]
χ_bus2 -0.055566 [MHz]
1/T1bus2 6.617660 [Hz]
T1bus2 24050.034508 [us]
Bus-Bus Couplings
gbus1_2 92.521861 [MHz]
Pass number:  2
[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 5.857617 [GHz]
EC 350.388157 [MHz]
EJ 13.847085 [GHz]
alpha -411.728650 [MHz]
dispersion 92.287308 [KHz]
Lq 0.000055 [nH]
Cq 55.282200 [fF]
T1 207.725217 [us]

**Coupling Properties**

tCqbus1 -3.405416 [fF]
gbus1_in_MHz -59.851833 [MHz]
χ_bus1 -0.891433 [MHz]
1/T1bus1 766.145413 [Hz]
T1bus1 207.734642 [us]

tCqbus2 0.032545 [fF]
gbus2_in_MHz 0.495547 [MHz]
χ_bus2 -0.000300 [MHz]
1

fQ          EC         EJ       alpha  dispersion  \
1  5.903517  356.312793  13.847085 -419.451115  106.412352   
2  5.857617  350.388157  13.847085  -411.72865   92.287308   
3   5.75881  337.829228  13.847085 -395.443152   67.388688   
4  5.656869  325.148902  13.847085 -379.115538   48.151031   
5  5.594406  317.516931  13.847085 -369.343684   38.947413   
6  5.563454  313.773688  13.847085 -364.565998   34.998313   
7  5.532484  310.053642  13.847085 -359.827728   31.409268   
8  5.515036  307.969088  13.847085 -357.176871   29.535818   

                                          gbus  \
1       [-62.05666653548034, 6.32263277965494]   
2    [-59.851833245320364, 0.4955468412122127]   
3   [-57.982123766711446, -0.8160965919759733]   
4  [-58.236694225520196, -0.17077017542435144]   
5  [-58.304481783023924, -0.09325930614370168]   
6   [-58.75575626672229, -0.09337209606599023]   
7   [-58.89658310217518, -0.09656236551073652]   
8   [-59.00045510111853, 0.031232928349868756]   

                                       chi_in_MHz    χr MHz     gr MHz  
1     [-1.0223639330492031, -0.05556612143303772]  1.022364  62.056667  
2    [-0.891432752696988, -0.0002999972766615658]  0.891433  59.851833  
3   [-0.7302934694284114, -0.0006288620961102058]  0.730293  57.982124  
4  [-0.6431329313829047, -2.1611574189766714e-05]  0.643133  58.236694  
5  [-0.5943184054871474, -5.6097594706627675e-06]  0.594318  58.304482  
6   [-0.5800343747704108, -5.261754473779534e-06]  0.580034  58.755756  
7   [-0.5602774810201381, -5.272925214780364e-06]  0.560277  58.896583  
8   [-0.5499769622136379, -5.321080300327384e-07]  0.549977  59.000455

In [44]:
fourq_q3d.plot_convergence_main(dict_lom)
fourq_q3d.plot_convergence_chi(dict_lom)

INFO 10:37AM [hfss_report_full_convergence]: Creating report for variation 0


Design "Q3DDesign2" info:
	# eigenmodes    0
	# variations    1


<Figure size 990x385 with 2 Axes>

In [24]:
fourq_q3d.disconnect_ansys()